# Q-Learning
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/


In [1]:
!pip install cmake 'gym[atari]' scipy

     |████████████████████████████████| 17.6 MB 11.6 MB/s eta 0:00:01    |██████████████                  | 7.7 MB 11.6 MB/s eta 0:00:01     |██████████████████▊             | 10.3 MB 11.6 MB/s eta 0:00:01     |████████████████████████▏       | 13.3 MB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 11.5 MB/s eta 0:00:01     |████████                        | 389 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 8.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.1 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 28.2 MB 84 kB/s  eta 0:00:01    |                                | 92 kB 11.9 MB/s eta 0:00:03     |██████████▎                     | 9.0 MB 9.1 MB/s eta 0:00:03     |█████████████████▌              | 15.4 MB 9.1 MB/s eta 0:00:02
     |████████████████████████████████| 829 kB 9.5 MB/s eta 0:00:01
  Created wheel for gym: filename=gym-0.17.2-

### This is the Gym taxi symulaton

In [4]:
import gym

env = gym.make("Taxi-v3").env

env.reset()

env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



Each cell has a state, taxi row, taxi col, passenger index - this is the position of the waiting passenger (Blue letter) and destination - magenta letter. The indexes goes in index order.[0,0] = 1, [0,4] = 2, [4,0] = 3, [4,4] = 4

In [48]:
state = env.encode(3, 1, 0, 2) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 322
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)


## Reward Table
Reward table is a martix of dimensions [states x actions]. The row 328 will show all the actions 
    {action: [(probability, nextstate, reward, done)]}

In [13]:
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [23]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 481
Penalties incurred: 145


In [24]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 481
State: 0
Action: 5
Reward: 20


## Reinforcement Learning
### Q-Learning
Uses the environment rewardsto find out the Quality of each move and over time find out the best action to take in a given state.

In our Taxi environment, we have the reward table, P, that the agent will learn from. It does thing by looking receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

A Q-value for a particular state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.

## Q-Table
An array with a row for every state and a column for every action, it is initialised to 0 and values updated during training. Q table has same dimensions as the reward table (P)

In [25]:
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [26]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

CPU times: user 45.8 s, sys: 175 ms, total: 46 s
Wall time: 48.7 s


In [27]:
q_table[328]

array([ -2.39921849,  -2.27325184,  -2.41539634,  -2.35892335,
       -10.41168971, -10.61212205])

In [38]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 1

frames = [] # for animation
for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1
        # Put each rendered frame into dict for animation
        frames.append({
            'frame': env.render(mode='ansi'),
            'state': state,
            'action': action,
            'reward': reward
            }
        )
        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 1 episodes:
Average timesteps per episode: 14.0
Average penalties per episode: 0.0


In [40]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 14
State: 410
Action: 5
Reward: 20
